In [1]:
from keras.models import load_model
import keras
from keras import backend as K
from scipy.spatial.distance import dice
import numpy as np
import re
import os
import glob
import nibabel as nib
from skimage.transform import resize
from skimage import img_as_bool
import matplotlib.pyplot as plt

Using TensorFlow backend.
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
smooth = 1

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


# TODO:
# die loss funktion geht eigentlich von 1 bis 0?, hier wird einfach einfach der negative DC verwendet 
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

In [4]:
from isles_data import load_imgs

P = "/home/lasse/Dokumente/isles18/TRAINING"

imgs, gts, add = load_imgs(P, 128)





/home/lasse/Dokumente/isles18/TRAINING/case_1/SMIR.Brain.XX.O.CT.*/*.nii


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 78: invalid start byte

In [3]:






ATH = "/home/lasse/Dokumente/isles18/TRAINING/case_{}/SMIR.Brain.XX.O.{}.*/*.nii"
CASES = range(int(95*0.75), 95)
MODALITIES = ("CT", "CT_CBF", "CT_CBV",  "CT_MTT", "CT_Tmax", "OT")

x_test_case = []
y_test_case = []
additional_data = []  # [case]["case" / "id" / "header"]

for c in CASES:
    img_mod = []
        
    for mod in MODALITIES:
        img_path = glob.glob(PATH.format(c, mod))[0]  # die Nifti-Datei wird unter Berücksichtigung von * gefunden
        img_file = nib.load(img_path)
        
        print("{}. {}".format(c, img_path))
        
        img = np.transpose(img_file.get_data())
        img = resize(img, (img.shape[0], 128, 128), mode="constant", anti_aliasing=False)
        
        img[img < 0] = 0  # Minuswerte zu 0 (im CT negative Hounsfield-Units für Fett)
        img /= img.max()  # Density-Range von 0-1    
        
        header = img_file.header
    
        if mod == "OT":
            y_test_case.append(np.expand_dims(img_as_bool(img), axis=3))
        else:
            img_mod.append(img)
        
        if mod == "CT_MTT":
            num = re.findall(r"\d+", img_path)
            additional_data.append({"case" : num[1], "id" : num[3], "header" : header})

    x_test_case.append(np.stack(img_mod, axis=3))    

71. /home/lasse/Dokumente/isles18/TRAINING/case_71/SMIR.Brain.XX.O.CT.346040/SMIR.Brain.XX.O.CT.346040.nii
71. /home/lasse/Dokumente/isles18/TRAINING/case_71/SMIR.Brain.XX.O.CT_CBF.346041/SMIR.Brain.XX.O.CT_CBF.346041.nii
71. /home/lasse/Dokumente/isles18/TRAINING/case_71/SMIR.Brain.XX.O.CT_CBV.346042/SMIR.Brain.XX.O.CT_CBV.346042.nii
71. /home/lasse/Dokumente/isles18/TRAINING/case_71/SMIR.Brain.XX.O.CT_MTT.346043/SMIR.Brain.XX.O.CT_MTT.346043.nii
71. /home/lasse/Dokumente/isles18/TRAINING/case_71/SMIR.Brain.XX.O.CT_Tmax.346045/SMIR.Brain.XX.O.CT_Tmax.346045.nii
71. /home/lasse/Dokumente/isles18/TRAINING/case_71/SMIR.Brain.XX.O.OT.346044/SMIR.Brain.XX.O.OT.346044.nii
72. /home/lasse/Dokumente/isles18/TRAINING/case_72/SMIR.Brain.XX.O.CT.346047/SMIR.Brain.XX.O.CT.346047.nii
72. /home/lasse/Dokumente/isles18/TRAINING/case_72/SMIR.Brain.XX.O.CT_CBF.346048/SMIR.Brain.XX.O.CT_CBF.346048.nii
72. /home/lasse/Dokumente/isles18/TRAINING/case_72/SMIR.Brain.XX.O.CT_CBV.346049/SMIR.Brain.XX.O.CT_CB

/home/lasse/.local/lib/python3.6/site-packages/skimage/util/dtype.py:126: UserWarning: Possible sign loss when converting negative image of type float64 to positive image of type bool.
  .format(dtypeobj_in, dtypeobj_out))
/home/lasse/.local/lib/python3.6/site-packages/skimage/util/dtype.py:130: UserWarning: Possible precision loss when converting from float64 to bool
  .format(dtypeobj_in, dtypeobj_out))


76. /home/lasse/Dokumente/isles18/TRAINING/case_76/SMIR.Brain.XX.O.OT.346079/SMIR.Brain.XX.O.OT.346079.nii
77. /home/lasse/Dokumente/isles18/TRAINING/case_77/SMIR.Brain.XX.O.CT.346082/SMIR.Brain.XX.O.CT.346082.nii
77. /home/lasse/Dokumente/isles18/TRAINING/case_77/SMIR.Brain.XX.O.CT_CBF.346083/SMIR.Brain.XX.O.CT_CBF.346083.nii
77. /home/lasse/Dokumente/isles18/TRAINING/case_77/SMIR.Brain.XX.O.CT_CBV.346084/SMIR.Brain.XX.O.CT_CBV.346084.nii
77. /home/lasse/Dokumente/isles18/TRAINING/case_77/SMIR.Brain.XX.O.CT_MTT.346085/SMIR.Brain.XX.O.CT_MTT.346085.nii
77. /home/lasse/Dokumente/isles18/TRAINING/case_77/SMIR.Brain.XX.O.CT_Tmax.346087/SMIR.Brain.XX.O.CT_Tmax.346087.nii
77. /home/lasse/Dokumente/isles18/TRAINING/case_77/SMIR.Brain.XX.O.OT.346086/SMIR.Brain.XX.O.OT.346086.nii
78. /home/lasse/Dokumente/isles18/TRAINING/case_78/SMIR.Brain.XX.O.CT.346089/SMIR.Brain.XX.O.CT.346089.nii
78. /home/lasse/Dokumente/isles18/TRAINING/case_78/SMIR.Brain.XX.O.CT_CBF.346090/SMIR.Brain.XX.O.CT_CBF.346090

89. /home/lasse/Dokumente/isles18/TRAINING/case_89/SMIR.Brain.XX.O.CT_MTT.346169/SMIR.Brain.XX.O.CT_MTT.346169.nii
89. /home/lasse/Dokumente/isles18/TRAINING/case_89/SMIR.Brain.XX.O.CT_Tmax.346171/SMIR.Brain.XX.O.CT_Tmax.346171.nii
89. /home/lasse/Dokumente/isles18/TRAINING/case_89/SMIR.Brain.XX.O.OT.346170/SMIR.Brain.XX.O.OT.346170.nii
90. /home/lasse/Dokumente/isles18/TRAINING/case_90/SMIR.Brain.XX.O.CT.346173/SMIR.Brain.XX.O.CT.346173.nii
90. /home/lasse/Dokumente/isles18/TRAINING/case_90/SMIR.Brain.XX.O.CT_CBF.346174/SMIR.Brain.XX.O.CT_CBF.346174.nii
90. /home/lasse/Dokumente/isles18/TRAINING/case_90/SMIR.Brain.XX.O.CT_CBV.346175/SMIR.Brain.XX.O.CT_CBV.346175.nii
90. /home/lasse/Dokumente/isles18/TRAINING/case_90/SMIR.Brain.XX.O.CT_MTT.346176/SMIR.Brain.XX.O.CT_MTT.346176.nii
90. /home/lasse/Dokumente/isles18/TRAINING/case_90/SMIR.Brain.XX.O.CT_Tmax.346178/SMIR.Brain.XX.O.CT_Tmax.346178.nii
90. /home/lasse/Dokumente/isles18/TRAINING/case_90/SMIR.Brain.XX.O.OT.346177/SMIR.Brain.XX.O

In [14]:
def test_model(model_path):
    model_name = os.path.splitext(os.path.basename(model_path))[0]
    output_path = os.path.join("./output/", model_name)
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    else:
        for f in glob.glob(os.path.join(output_path, "*")):
            os.remove(f)

    print("Lade Model...")
    model = load_model(model_path, custom_objects={"dice_coef" : dice_coef, "dice_coef_loss" : dice_coef_loss})

    dcs = []

    for x, y, a in zip(x_test_case, y_test_case, additional_data):

        # verbose = 0, da Keras hier keine relevanten Inforamtion liefert
        img = model.predict(x, verbose=0).squeeze()

        dc = 1.0 - dice(img.flatten(), y.flatten())
        dcs.append(dc)
        print("{}({}): {:0.2f}".format(a["case"], a["id"], dc))

        #plt.imshow(img[1, :, :])
        #plt.show()

        final_img = np.transpose(img)
        final_img = resize(final_img, (256, 256,  final_img.shape[2]), mode="constant", anti_aliasing=False).astype(np.bool)

        nimg = nib.Nifti1Image(final_img, np.eye(4), header=a["header"])
        nib.save(nimg, os.path.join(output_path, "SMIR.UNET2.{}.nii".format(a["id"])))

    print("DC: {}".format(np.mean(dcs)))
    
test_model("./cloud_models/unet-gcloud-dl.hd5f")

Lade Model...
71(346043): 0.84
72(346050): 0.72
73(346057): 0.77
74(346064): 0.00
75(346071): 0.56
76(346078): 0.62
77(346085): 0.79
78(346092): 0.22
79(346099): 0.18
80(346106): 0.01
81(346113): 0.25
82(346120): 0.72
83(346127): 0.37
84(346134): 0.67
85(346141): 0.65
86(346148): 0.58
87(346155): 0.54
88(346162): 0.18
89(346169): 0.07
90(346176): 0.51
91(346183): 0.37
92(346190): 0.34
93(346197): 0.73
94(346204): 0.00
DC: 0.4455382209921985
